<a href="https://colab.research.google.com/github/tzlilLV97/Deep-Learning-Final-Project/blob/main/V1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Final Project: 

Deep learning Course

Tzlil Lev or 318646510

Alon Feldman - too old to have id number

Project Name : Predict League of Legends victory 







In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import classification_report
import xgboost as xgb
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from google.colab import drive
drive.mount('/content/gdrive')

# Path to save the dataset.
PATH_TO_DATA = '/content/gdrive/MyDrive/Colab Notebooks/data/games.csv' 

Mounted at /content/gdrive


Data Preperation 


In [9]:

def data_preparation(df_first):
    df = df_first.iloc[:, [2, 5, 6, 7, 8, 9, 10, 11, 14, 17, 20, 23, 26, 27, 28, 29, 30, 36, 39, 42, 45, 48,  51, 52, 53, 54, 55]]
    ## add 2, 5
    ## NORMALIZE THE GAME DURATION
    df['gameDuration'] = df['gameDuration'].apply(lambda x: np.log(x))
    # print(df['gameDuration'])
    labels = df_first.iloc[:, 4].to_numpy() - 1
    df = df.to_numpy()
    return df, labels


def get_batch(data,labels, start, end):
    x = data[start:end]
    s = labels[start:end]
    return x, s

df = pd.read_csv(PATH_TO_DATA)
data, labels = data_preparation(df)
n = len(data)
len_train = int(0.8 * n)
len_test = (n-int(len_train))//2
len_val = len_test
train, valid, test = (data[:len_train],labels[:len_train]), (data[len_train:len_train+len_test],labels[len_train:len_train+len_test]), (data[len_train+len_test:],labels[len_train+len_test:])




<ipython-input-9-47b5147fee2e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['gameDuration'] = df['gameDuration'].apply(lambda x: np.log(x))


**BENCHMARK**

In order to evalute our model, we compared it to different models, to 

observe the results and compare them to our network. 

We choose the following models:


* XGBoost

* Random Forest

* KNN

* SVM

* Random Forest

The Following Code is the implementation of each of them


In [10]:
### SVM 

def estimate_accuracy_svm(model, data, labels, batch_size=5000, max_N=100000):
    """
    Estimate the accuracy of the model on the data. To reduce
    computation time, use at most `max_N` elements of `data` to
    produce the estimate.
    """
    correct = 0
    N = 0
    for i in range(0, len(data), batch_size):
        # get a batch of data
        xt, st = get_batch(data, labels, i, i + batch_size)
        # forward pass prediction
        y = model.predict(xt)
        pred = np.squeeze(y)
        correct += np.sum(pred==st)
        N += len(st)

        if N > max_N:
            break
    return correct / N



def svmModel(train, valid, test):
    x_train, y_train = train
    x_valid, y_valid = valid
    x_test, y_test = test
    svm_model = svm.SVC()
    svm_model.fit(x_train, y_train)
    y_pred = svm_model.predict(x_test)
    print("SVM Accuracy on validation set: {:.2f}%".format(estimate_accuracy_svm(svm_model, x_valid, y_valid) * 100))
    print("SVM Accuracy on test set: {:.2f}%".format(estimate_accuracy_svm(svm_model, x_test, y_test) * 100))


svmModel(train, valid, test)
         



SVM Accuracy on validation set: 94.08%
SVM Accuracy on test set: 93.88%


In [11]:
### KNN
def estimate_accuracy_KNN(model, data, labels, batch_size=5000, max_N=100000):
    """
    Estimate the accuracy of the model on the data. To reduce
    computation time, use at most `max_N` elements of `data` to
    produce the estimate.
    """

    correct = 0
    N = 0
    for i in range(0, len(data), batch_size):
        # get a batch of data
        xt, st = get_batch(data, labels, i, i + batch_size)
        # forward pass prediction
        y = model.predict(xt)
        pred = np.squeeze(y)
        correct += np.sum(pred==st)
        N += len(st)
        if N > max_N:
            break
    return correct / N

def KNN(train,valid,test):
    x_train, y_train = train
    x_valid, y_valid = valid
    x_test, y_test = test
    model = KNeighborsClassifier(n_neighbors=5)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print("KNN Accuracy on validation set: {:.2f}%".format(estimate_accuracy_KNN(model, x_valid, y_valid) * 100))
    print("KNN Accuracy on test set: {:.2f}%".format(estimate_accuracy_KNN(model, x_test, y_test) * 100))

KNN(train, valid, test)

KNN Accuracy on validation set: 50.42%
KNN Accuracy on test set: 52.22%


In [19]:
### XGBoost

def compare_xgb_predictions(xgb_model, X, y):
    # Make predictions using the trained model
    global n, y_test,x_test,x_valid,y_valid
    xgb_predictions = xgb_model.predict(X)

    # Round the predictions to 0 or 1
    xgb_predictions = [round(pred) for pred in xgb_predictions]

    # Compare the predictions with the real labels
    correct = 0
    total = len(X)
    for pred, label in zip(xgb_predictions, y):
        if pred == label:
            correct += 1
    # Calculate the accuracy of the predictions
    accuracy = correct / total
    return accuracy



  
def xgb_classifier(train,valid,test):
    global n, y_test,x_test,x_valid,y_valid
    x_train, y_train = train
    x_valid, y_valid = valid
    x_test, y_test = test
    #  Write your code here
    model = xgb.XGBClassifier()
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
  #  print(classification_report(y_test, y_pred))
    print("XGB Accuracy on validation set: {:.2f}%".format(compare_xgb_predictions(model, x_valid, y_valid) * 100))
    print("XGB Accuracy on test set: {:.2f}%".format(compare_xgb_predictions(model, x_test, y_test) * 100))
          #"compare_xgb_predictions(model, x_test, y_test))

xgb_classifier(train, valid, test)

XGB Accuracy on validation set: 97.28%
XGB Accuracy on test set: 97.30%


Random Forest

In [ ]:
def random_forest(train_data, validation_data, test_data):
    # Create a random forest classifier
    clf = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=0)

    # Train the classifier
    clf.fit(train_data[0], train_data[1])

    # Predict the labels of the test set
    preds = clf.predict(validation_data[0])

    # Compute the accuracy: accuracy
    accuracy = accuracy_score(validation_data[1], preds)
    print("Random Forest Accuracy on Validation Set: %.2f%%" % (accuracy * 100))
    print("Random Forest Accuracy on Training Set: %.2f%%" % (accuracy_score(train_data[1], clf.predict(train_data[0])) * 100))
    print("Random Forest Accuracy on Test Set: %.2f%%" % (accuracy_score(test_data[1], clf.predict(test_data[0])) * 100))

random_forest(train, valid, test)

Accuracy estimation 

In [ ]:



def estimate_accuracy_torch(model, data, labels, batch_size=5000, max_N=100000):
  """
  Estimate the accuracy of the model on the data. To reduce
  computation time, use at most `max_N` elements of `data` to
  produce the estimate.
  """

  correct = 0
  N = 0
  for i in range(0, len(data), batch_size):
      # get a batch of data
      xt, st = get_batch(data, labels, i, i + batch_size)
      # forward pass prediction
      y = model(torch.Tensor(xt))
      y = y.detach().numpy()  # convert the PyTorch tensor => numpy array
      y = np.where(y > 0.5, 1, 0)
      pred = np.squeeze(y)
      correct += np.sum(pred==st)
      N += len(st)

      if N > max_N:
          break
  return correct / N




  

Neural Network

In [ ]:
class PyTorchMLP(nn.Module):
    def _init_(self, num_hidden=5):
        super(PyTorchMLP, self)._init_()
        self.input_size = 27  # input_size
        self.hidden_size = 50 # hidden_size was 50
        self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(self.hidden_size, self.hidden_size)
        self.fc3 = torch.nn.Linear(self.hidden_size, self.hidden_size)
        self.fc4 = torch.nn.Linear(self.hidden_size, self.hidden_size)
        self.fc5 = torch.nn.Linear(self.hidden_size, self.hidden_size)
        self.fc6 = torch.nn.Linear(self.hidden_size, 1)
       # self.dropout = torch.nn.Dropout(0.1)
       # self.optimizer2 = torch.optim.RMSprop(self.parameters())
       #  self.sigmoid = F.sigmoid()
       #  self.tanh = F.tanh()
        self.normalizer = torch.nn.GroupNorm(1, self.hidden_size)
    def forward(self, inp):
        hidden1 = self.fc1(inp)
        relu1 = F.relu(hidden1)
  #      bypass = self.fc6(relu1)
        hidden2 = self.fc2(relu1)
        # Add bypass branch
        bypass = self.fc6(hidden1)
        relu3 = F.sigmoid(hidden2)
        output = self.fc6(relu3)
        # Combine bypass and regular outputs
        #output = F.sigmoid(bypass+ output)
        output = F.sigmoid(output)
        return output
    # def forward(self, inp):
    #     hidden1 = self.fc1(inp)
    #     relu1 = self.relu(hidden1)
    #     ##ADD BYPASS BRANCH
    #     bypass = self.fc6(relu1)
    #     hidden2 = self.fc2(relu1)
    #     relu2 = self.relu(hidden2)
    #     hidden3 = self.fc3(relu2)
    #     relu3 = self.relu(hidden3)
    #     output = self.fc6(relu3)
    #     ##COMBINE BYPASS AND REGULAR OUTPUTS
    #     return self.sigmoid(output+bypass)


NameError: ignored

In [ ]:
def estimate_accuracy_torch(model, data, labels, batch_size=5000, max_N=100000):
    """
    Estimate the accuracy of the model on the data. To reduce
    computation time, use at most `max_N` elements of `data` to
    produce the estimate.
    """

    correct = 0
    N = 0
    for i in range(0, len(data), batch_size):
        # get a batch of data
        xt, st = get_batch(data, labels, i, i + batch_size)
        # forward pass prediction
        y = model(torch.Tensor(xt))
        y = y.detach().numpy()  # convert the PyTorch tensor => numpy array
        y = np.where(y > 0.5, 1, 0)
        pred = np.squeeze👍
        correct += np.sum(pred==st)
        N += len(st)

        if N > max_N:
            break
    return correct / N

def plot_learning_curve(iters, losses, iters_sub, train_accs, val_accs):
  """
  Plot the learning curve.
  """
  plt.title("Learning Curve: Loss per Iteration")
  plt.plot(iters, losses, label="Train")
  plt.xlabel("Iterations")
  plt.ylabel("Loss")
  plt.show()

  plt.title("Learning Curve: Accuracy per Iteration")
  plt.plot(iters_sub, train_accs, label="Train")
  plt.plot(iters_sub, val_accs, label="Validation")
  plt.xlabel("Iterations")
  plt.ylabel("Accuracy")
  plt.legend(loc='best')
  plt.show()

def make_prediction_torch(model, game_data):

  b = game_data
  toBePredicted = torch.Tensor(b)
  print(toBePredicted)
  output = model(toBePredicted)
  print(output)
  #  Write your code here

def pytorch_gradient_descent(model, train_data,
                                validation_data,test_data,
                                batch_size=100,
                                learning_rate=0.001,
                                weight_decay=0,
                                epochs=1000):
  ##COSTS
  #criterion =nn.BCELoss()# nn.CrossEntropyLoss()
  #criterion = nn.NLLLoss()
  criterion = nn.MSELoss()
  #criterion = nn.CrossEntropyLoss()
  ##EXTRA FEATURES
  # droper = nn.Dropout(p=0.1)
  droper = 0

  ## OPTIMIZER
  #optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
  optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
  iters, losses = [], []
  iters_sub, train_accs, val_accs = [], [], []

  n = 0  # the number of iterations
  while True:
      for i in range(0, train_data[0].shape[0], batch_size):
          if (i + batch_size) > train_data[0].shape[0]:
              break
          # get the input and targets of a minibatch
          xt, st = get_batch(train_data[0],train_data[1], i, i + batch_size)
          ## Create normalized data
          # xt,mio,sigma = probabilistic_normalization(xt)
          # convert from numpy arrays to PyTorch tensors
          xt = torch.Tensor(xt)
          st = st
          st = torch.Tensor(st).float().unsqueeze(1)
          if droper:
              zs = model(droper(xt)) # compute prediction logit, use dropout if wanted
          else:
              zs = model(xt)
          loss = criterion(st, zs)  # compute the total loss
          loss.backward()  # compute updates for each parameter
          optimizer.step()  # make the updates for each parameter
          optimizer.zero_grad()  # a clean up step for PyTorch
          # save the current training information
          iters.append👎
          losses.append(float(loss) / batch_size)  # compute average loss

          if n % 10 == 0:
          #    print(classification_report(zs.detach().numpy(),st.detach().numpy()))
          #    continue
              iters_sub.append👎
              train_cost = float(loss.detach().numpy())
              train_acc = estimate_accuracy_torch(model, train_data[0],train_data[1])
              train_accs.append(train_acc)
              val_acc = estimate_accuracy_torch(model, validation_data[0], validation_data[1])
              val_accs.append(val_acc)
          #    print("Iter %d. [Val Acc %.0f%%] [Train Acc %.0f%%, Loss %f]" % (
          #        n, val_acc * 100, train_acc * 100, train_cost))

          # increment the iteration number
      n += 1

      if n > epochs:
          train_cost = float(loss.detach().numpy())
          train_acc = estimate_accuracy_torch(model, train_data[0], train_data[1])
          train_accs.append(train_acc)
          val_acc = estimate_accuracy_torch(model,validation_data[0], validation_data[1])
          val_accs.append(val_acc)
          # print("Iter %d. [Val Acc %.0f%%] [Train Acc %.0f%%, Loss %f]" % (
          #     n, val_acc * 100, train_acc * 100, train_cost))
          print("NN Accuracy on Validation Set: %.2f%%" % (val_acc * 100))
          print("NN Accuracy on Training Set: %.2f%%" % (train_acc * 100))
          print("NN Accuracy on Test Set: %.2f%%" % (estimate_accuracy_torch(model, test_data[0], test_data[1]) * 100))
          return iters, losses, iters_sub, train_accs, val_accs


In [ ]:
pytorch_mlp = PyTorchMLP()
learning_curve_info = pytorch_gradient_descent(pytorch_mlp, train,valid ,test,batch_size=5000,
                            learning_rate=0.00013,
                            weight_decay=0.0000000,
                            epochs=400)